source: https://realpython.com/pandas-groupby/

In [1]:
import pandas as pd

In [2]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}

In [3]:
df = pd.read_csv(
    "legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

In [4]:
df.head()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


In [5]:
df.tail()

,last_name,first_name,birthday,gender,type,state,party
11976,Collins,Chris,1950-05-20,M,rep,NY,Republican
11977,Cummings,Elijah,1951-01-18,M,rep,MD,Democrat
11978,Hill,Katie,1987-08-25,F,rep,CA,Democrat
11979,Isakson,John,1944-12-28,M,sen,GA,Republican
11980,Hunter,Duncan,1976-12-07,M,rep,CA,Republican


In [6]:
df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

**SELECT state, count(name)
FROM df
GROUP BY state
ORDER BY state;**


In [7]:
#n_by_state = df.groupby("state").count()
#type(n_by_state)
#print(n_by_state)
n_by_state = df.groupby("state")["last_name"].count()
#print(n_by_state)

You call .groupby() and pass the name of the column you want to group on, which is "state". Then, you use ["last_name"] to specify the columns on which you want to perform the actual aggregation.

**Here’s an example of grouping jointly on two columns, which finds the count of Congressional members broken out by state and then by gender:**

SELECT state, gender, count(name)
FROM df
GROUP BY state, gender
ORDER BY state, gender;


In [8]:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     M          16
AL     F           3
       M         203
AR     F           5
       M         112
                ... 
WI     M         197
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 104, dtype: int64

In the Pandas version, the grouped-on columns are pushed into the MultiIndex of the resulting Series by default:

In [9]:
n_by_state_gender = df.groupby(["state", "gender"])["last_name"].count()
type(n_by_state_gender)

pandas.core.series.Series

In [10]:
n_by_state_gender.index[:5]

MultiIndex([('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F'),
            ('AR', 'M')],
           names=['state', 'gender'])

In [11]:
n_by_state_gender

state  gender
AK     M          16
AL     F           3
       M         203
AR     F           5
       M         112
                ... 
WI     M         197
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 104, dtype: int64

To more closely emulate the SQL result and push the grouped-on columns back into columns in the result, you an use as_index=False:

In [12]:
tmp_df = df.groupby(["state", "gender"], as_index=False)["last_name"].count()
print(type(tmp_df))
tmp_df

<class 'pandas.core.frame.DataFrame'>


,state,gender,last_name
0,AK,F,NaN
1,AK,M,16.0
2,AL,F,3.0
3,AL,M,203.0
4,AR,F,5.0
...,...,...,...
111,WI,M,197.0
112,WV,F,1.0
113,WV,M,119.0
114,WY,F,2.0


This produces a DataFrame with three columns and a RangeIndex, rather than a Series with a MultiIndex. In short, using as_index=False will make your result more closely mimic the default SQL output for a similar operation.

Also note that the SQL queries above explicitly use ORDER BY, whereas .groupby() does not. That’s because .groupby() does this by default through its parameter sort, which is True unless you tell it otherwise:


In [13]:
# Don't sort results by the sort keys
df.groupby("state", sort=False)["last_name"].count()

state
DE      97
VA     432
SC     251
MD     306
PA    1053
MA     426
NJ     359
GA     310
NY    1462
NC     354
CT     240
VT     115
KY     373
RI     107
NH     181
TN     299
OH     674
MS     155
OL       2
IN     341
LA     197
IL     486
MO     333
AL     206
AR     117
ME     175
FL     155
MI     294
IA     202
WI     197
TX     256
CA     363
OR      89
MN     160
NM      54
NE     127
WA      95
KS     141
UT      53
NV      56
CO      90
WV     120
DK       9
AZ      48
ID      59
MT      52
WY      40
DC       2
ND      44
SD      51
OK      92
HI      23
PR      19
AK      16
PI      13
VI       4
GU       4
AS       2
Name: last_name, dtype: int64

In [14]:
by_state = df.groupby("state")
print(by_state)

What is that DataFrameGroupBy thing? Its .__str__() doesn’t give you much information into what it actually is or how it works. The reason that a DataFrameGroupBy object can be difficult to wrap your head around is that it’s lazy in nature. It doesn’t really do any operations to produce a useful result until you say so.

One term that’s frequently used alongside .groupby() is split-apply-combine. This refers to a chain of three steps:

    Split a table into groups
    Apply some operations to each of those smaller tables
    Combine the results

It can be difficult to inspect df.groupby("state") because it does virtually none of these things until you do something with the resulting object. Again, a Pandas GroupBy object is lazy. It delays virtually every part of the split-apply-combine process until you invoke a method on it.

So, how can you mentally separate the split, apply, and combine stages if you can’t see any of them happening in isolation? One useful way to inspect a Pandas GroupBy object and see the splitting in action is to iterate over it. This is implemented in DataFrameGroupBy.__iter__() and produces an iterator of (group, DataFrame) pairs for DataFrames:

In [15]:
for state, frame in by_state:
    print(f"First 2 entries for {state!r}")
    print("-----------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
-----------------
     last_name first_name   birthday gender type state        party
6619    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6647      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
-----------------
    last_name first_name   birthday gender type state       party
912   Crowell       John 1780-09-18      M  rep    AL  Republican
991    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
-----------------
     last_name first_name   birthday gender type state party
1001     Bates      James 1788-08-25      M  rep    AR   NaN
1279    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
-----------------
          last_name first_name   birthday gender type state     party
10797         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11755  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

First 2 entries for 'AZ'
----

         last_name first_name   birthday gender type state party
9            Floyd    William 1734-12-17      M  rep    NY   NaN
26  Van Rensselaer   Jeremiah 1738-08-27      M  rep    NY   NaN

First 2 entries for 'OH'
-----------------
    last_name first_name   birthday gender type state       party
226  McMillan    William 1764-03-02      M  rep    OH         NaN
254   Fearing       Paul 1762-02-28      M  rep    OH  Federalist

First 2 entries for 'OK'
-----------------
     last_name first_name   birthday gender type state        party
5601    Harvey      David 1845-03-20      M  rep    OK   Republican
6060  Callahan      James 1852-12-19      M  rep    OK  Free Silver

First 2 entries for 'OL'
-----------------
    last_name first_name   birthday gender type state party
404     Clark     Daniel        NaT      M  rep    OL   NaN
503   Poydras     Julien 1740-04-03      M  rep    OL   NaN

First 2 entries for 'OR'
-----------------
     last_name first_name   birthday gender typ

If you’re working on a challenging aggregation problem, then iterating over the Pandas GroupBy object can be a great way to visualize the split part of split-apply-combine.

There are a few other methods and properties that let you look into the individual groups and their splits. The .groups attribute will give you a dictionary of {group name: group label} pairs. For example, by_state is a dict with states as keys. Here’s the value for the "PA" key:

In [36]:
#by_state.describe()
by_state

In [16]:
by_state.groups['PA']

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11842, 11866, 11875, 11877, 11887, 11891, 11932, 11945, 11959,
            11973],
           dtype='int64', length=1053)

Each value is a sequence of the index locations for the rows belonging to that particular group. In the output above, 4, 19, and 21 are the first indices in df at which the state equals “PA.”

You can also use .get_group() as a way to drill down to the sub-table from a single group:

In [28]:
by_state.get_group('PA')

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


This is virtually equivalent to using .loc[]. You could get the same output with something like 
  
  df.loc[df["state"] == "PA"].

In [45]:
df.loc[df["state"] == 'NY'].sort_values(by='birthday', ascending=False)[:20]

,last_name,first_name,birthday,gender,type,state,party
11794,Grimm,Michael,1970-02-07,M,rep,NY,Republican
11619,Murphy,Scott,1970-01-26,M,rep,NY,Democrat
11786,Maffei,Daniel,1968-07-04,M,rep,NY,Democrat
11477,Fossella,Vito,1965-03-09,M,rep,NY,Republican
11632,Weiner,Anthony,1964-09-04,M,rep,NY,Democrat
11826,Gibson,Christopher,1964-05-13,M,rep,NY,Republican
11629,Lee,Christopher,1964-04-01,M,rep,NY,Republican
11894,Crowley,Joseph,1962-03-16,M,rep,NY,Democrat
11968,Tenney,Claudia,1961-02-04,F,rep,NY,Republican
11679,Hayworth,Nan,1959-12-14,F,rep,NY,Republican


Next, what about the apply part? You can think of this step of the process as applying the same operation (or callable) to every “sub-table” that is produced by the splitting stage. (I don’t know if “sub-table” is the technical term, but I haven’t found a better one 🤷‍♂️)

From the Pandas GroupBy object by_state, you can grab the initial U.S. state and DataFrame with next(). When you iterate over a Pandas GroupBy object, you’ll get pairs that you can unpack into two variables:

In [50]:
itr = iter(by_state)
print(type(itr))
state, frame = next(itr) # First tuple from iterator
print(state)
state, frame = next(itr) # Second tuple from iterator
print(state)
state, frame = next(itr) # Third tuple from iterator
print(state)

<class 'generator'>
AK
AL
AR


In [51]:
df.groupby("state")["last_name"].count()

state
AK      16
AL     206
AR     117
AS       2
AZ      48
CA     363
CO      90
CT     240
DC       2
DE      97
DK       9
FL     155
GA     310
GU       4
HI      23
IA     202
ID      59
IL     486
IN     341
KS     141
KY     373
LA     197
MA     426
MD     306
ME     175
MI     294
MN     160
MO     333
MS     155
MT      52
NC     354
ND      44
NE     127
NH     181
NJ     359
NM      54
NV      56
NY    1462
OH     674
OK      92
OL       2
OR      89
PA    1053
PI      13
PR      19
RI     107
SC     251
SD      51
TN     299
TX     256
UT      53
VA     432
VI       4
VT     115
WA      95
WI     197
WV     120
WY      40
Name: last_name, dtype: int64

The apply stage, when applied to your single, subsetted DataFrame, would look like this:

In [55]:
state, frame = next(iter(by_state))
print(state)
frame["last_name"].count()  # Count for state == 'AK'

AK


16

You can see that the result, 16, matches the value for AK in the combined result.

The last step, combine, is the most self-explanatory. It simply takes the results of all of the applied operations on all of the sub-tables and combines them back together in an intuitive way.